In [ ]:
### selenium 문법
0. find(단일 태그) / findall(복수 태그) // select(복수의 선택자: . #) / select_one(단일 선택자)
1. find_elements : 복수의 값을 찾아온다 -> 리스트 (동일한 요소를 전체 다 찾아서 가져온다)
2. find_element : 단수의 값을 찾아온다 -> 위에서 문서를 읽다가, 최초로 발견한 값을 찾아오고 종료
3. By : .TAG_NAME (태그를 중심), .ID (아이디중심) -> 문서 1번밖에 사용불가, .CSS_SELECTOR (선택자중심 -> 클래스, 태그, 아이디), .NAME / By.XPATH, Xpath 문법 활용 / By.CLASS_NAME
   
5. get_attribute() : 특정 속성의 속성값을 찾아오고자 할 때
6. .clear() : 특정 영역 내 값을 삭제 (Form 요소 기준) > imput[type=radio, check, tect], textarea 외
7. .send_keys() : 어떤 값 혹은 키보드의 키값을 전달
8. Keys : 키보드의 키를 입력하고자 할 때
9. sleep(N) : N초의 시간동안을 지연시키고자 할 때
10. screenshot(filename) : filename으로 스크린 캡처 가능
11. set_window_size(가로, 세로) : 윈도의 창을 가로 세로 사이즈로 조정

In [ ]:
# pseudo-code
# 무신사 사이트 접속
# 상단 검색바 클릭
# 인기 검색어 > 1등 키워드를 서칭
# 니트 키워드 입력 > 니트 상품 PLP
# 니트 관련 상품 목록 > 인기상품 5개 선정
# 인기 상품 5개 각각의 상세페이지 PDP 접속
# 각 상품별 리뷰(댓글) 목록 확인
# 해당 리뷰(댓글) 수집
# 엑셀 저장
# MySQL 저장

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A" 
service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") 
options.add_argument("--no-sandbox") 

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(3)

search_btn = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_btn.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text  
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)

print("타겟키워드 : ", target_keyword)
print("페이지타이틀 : ", driver.title)

driver.quit()

타겟키워드 :  니트
페이지타이틀 :  니트 | 무신사 추천 상품


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A" 
service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") 
options.add_argument("--no-sandbox") 

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

search_btn = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_btn.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text  
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)
time.sleep(2)

product_data = list()

for _ in range(2) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    products = driver.find_elements(By.CSS_SELECTOR, "div.sc-cpclqO.jrLXZA")[:5]
   # for product in products :
       # try : 
           # product_info = product.find_element(By.CSS_SELECTOR, "div.sc-itBLYH.cDYUnS")
           # brand = product_info.find_element(By.CSS_SELECTOR, "span.text-etc_11px_semibold").text.strip()
           # name = product_info.find_element(By.CSS_SELECTOR, "span.text-body_13px_reg").text.strip()
           # url = product_info.find_element(By.CSS_SELECTOR, "a.gtm-select-item").get_attribute("href")
           # review_infos = product_info.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")
           # for review_info in review_infos :
               # review_rating = review_info[0]
               # review_count = review_info[1]
               # product_data.append({"브랜드명" : brand, "상품명" : name, "URL" : url, "평점" : review_rating, "리뷰수" : review_count})
        # except :
           # continue
    for product in products :
        product_info = product.find_element(By.CSS_SELECTOR, "div.sc-itBLYH.cDYUnS")
        brand = product_info.find_element(By.CSS_SELECTOR, "span.text-etc_11px_semibold").text.strip()
        name = product_info.find_element(By.CSS_SELECTOR, "span.text-body_13px_reg").text.strip()
        url = product_info.find_element(By.CSS_SELECTOR, "a.gtm-select-item").get_attribute("href")
        #review_infos = product_info.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")
        #for review_info in review_infos :
            #review_rating = review_info[0]
            #review_count = review_info[1]
        product_data.append({"브랜드명" : brand, "상품명" : name, "URL" : url})
    

print(product_data)
driver.quit()

[{'브랜드명': '수아레', '상품명': '워셔블 하찌 니트 - 12 COLOR', 'URL': 'https://www.musinsa.com/products/2396645'}, {'브랜드명': '수아레', '상품명': '워셔블 케이블 라운드 니트 - 5 COLOR', 'URL': 'https://www.musinsa.com/products/3494992'}, {'브랜드명': '피지컬 디파트먼트', '상품명': '하트애플 크롭 니트_차콜', 'URL': 'https://www.musinsa.com/products/4344303'}, {'브랜드명': '커버낫', '상품명': '램스울 카라 니트 네이비', 'URL': 'https://www.musinsa.com/products/3487580'}, {'브랜드명': '키뮤어', '상품명': '[유튜버 PICK] 워셔블 카라니트 풀집업_5 COLOR', 'URL': 'https://www.musinsa.com/products/4348898'}, {'브랜드명': '비버리힐스 폴로클럽', '상품명': '여성 하프 집업 스웻 셔츠 3종택1 TS006', 'URL': 'https://www.musinsa.com/products/4582950'}, {'브랜드명': '라운드트리코', '상품명': '홀터 라운드넥 니트 - 블랙', 'URL': 'https://www.musinsa.com/products/3546644'}, {'브랜드명': '레터프롬문', '상품명': '원 숄더 레이어드 니트 탑 ( 멜란지그레이 )', 'URL': 'https://www.musinsa.com/products/4555007'}, {'브랜드명': '브이투', '상품명': '[SET]데일리 워셔블 소프트 하찌 니트 + 케이블 소프트 니트_8COLOR', 'URL': 'https://www.musinsa.com/products/4331963'}, {'브랜드명': '커즈넬로', '상품명': '멀티 블랜드 브이넥 니트_브라운', 'URL': 'https://w